In [1]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils.utils as ut
import utils.graph_templates

from fredapi import Fred

In [2]:
# Call the graph the exact same thing as its notebook (minus the ipynb suffix)
GRAPH_NAME = "r_star"

API_KEY_PATH = ut.get_repo_root() / "fred_api_key.txt" 

fred = Fred(api_key_file = API_KEY_PATH)

In [3]:
pio.templates.default = 'fed_2025'

# Now is a good time to set the path to the graph output folder!
GRAPH_OUTPUT_PATH = ut.get_repo_root() / "figures"

In [4]:
GRAPH_OUTPUT_PATH

WindowsPath('C:/Users/Bobby/Desktop/Fed Challenge 2025/NEUFed2025/figures')

In [5]:
today = dt.date.today()

hlb_df = pd.read_excel(ut.get_repo_root() / "code" / "data" / "hlb_2.xlsx")

r_star = hlb_df.set_index(pd.to_datetime(hlb_df["Date"]))['Natural Rate (r*)'].rename("LW Model R* (Quarterly)")

r_star.tail()


Date
2024-04-01    1.3647
2024-07-01    1.3657
2024-10-01    1.3671
2025-01-01    1.3730
2025-04-01    1.3730
Name: LW Model R* (Quarterly), dtype: float64

In [6]:
today = dt.date.today()

effr = fred.get_series(
    series_id="EFFR",
    observation_start=dt.date(2019, 1, 1),
    observation_end=today,
    frequency='m',
).rename("Fed Funds")

pce = fred.get_series(
    series_id="PCEPI",
    observation_start=dt.date(2019, 1, 1),
    observation_end=today,
    units='pc1'
).rename("PCE")

clv_fed = fred.get_series(
    series_id="REAINTRATREARAT1YE",
    observation_start=dt.date(2019, 1, 1),
    observation_end=today,
).rename("Cleveland Fed Estimated 1 Year Real Interest Rate")

real = (effr - pce).dropna().rename('Fisher Implied Real Rate ')

joined_df = pd.concat(
    [real, r_star, clv_fed],
    axis = 1,
    join='outer'
).fillna(method='ffill')

joined_df = joined_df[joined_df.index >= '2024-01-01']
joined_df


C:\Users\Bobby\AppData\Local\Temp\ipykernel_26132\680554228.py:29: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  ).fillna(method='ffill')


,Fisher Implied Real Rate,LW Model R* (Quarterly),Cleveland Fed Estimated 1 Year Real Interest Rate
2024-01-01,2.63125,1.3619,2.506889
2024-02-01,2.62494,1.3619,2.395777
2024-03-01,2.40206,1.3619,3.035620
2024-04-01,2.50385,1.3647,2.084637
2024-05-01,2.66833,1.3647,1.953609
2024-06-01,2.78163,1.3647,2.317161
2024-07-01,2.73562,1.3657,2.513466
2024-08-01,2.91701,1.3657,2.540358
2024-09-01,2.86923,1.3657,2.445496
2024-10-01,2.35231,1.3671,2.211509


5. Now that all our data is ready, make the graph and have it save itself as a .html file to graph_output whenver the notebooks is rerun. The name of the file should exactly match the notebook name. For instance, this file "example.ipynb" produces the graph "example.html." Nice work, you made a graph! 

In [7]:
import numpy as np

# First make the figure
fig = go.Figure()


# Loop the columns of the dataframe and plot each as a separate trace
for col in joined_df.columns:
    fig.add_trace(
        go.Scatter(
            x=joined_df.index,
            y=joined_df[col],
            mode='lines',
            name=col
        )
    )

# Update the titles, using the html tage <sup> for a subtitle 
fig.update_layout(
    title = dict(text = 'R* and Real Rates Estimates<br><sup>Monthly </sup>'),
    xaxis_title="Date",
    yaxis_title="Change (%)",
)

# This is graph specific, but here we want the y-axis to be percent signs 
fig.update_yaxes(
    tickformat=".2f%",
    ticksuffix="%",
)

# Again, graph specific, we have a mutliyear series and want tick marks to be years
fig.update_xaxes(
    type='date',
    dtick='M12',
    tickformat='%Y',
    title_text=None,
)

fig.update_layout(
    height=550,
    width=1150,
        legend=dict(
            orientation='v'
        )
)

fig.add_hline(
    y=1,
    line_dash="dash",
    line_color="red",
    annotation_text="Sep. SEP Fisher-Implied R*: 1%</br>",
    annotation_position="bottom left"
)

ut.add_end_labels(fig = fig, offset=0.05)

fig.update_layout(legend=dict(font=dict(size=20)))

# Show our figure (Dimensions may be off on different screen sizes)
fig.show()

# This should be the same for EVERY GRAPH!
# Save it to the graph_output folder with the name matching the file, as HTML
fig.write_html(GRAPH_OUTPUT_PATH / f"{GRAPH_NAME}.html")